# How does the router node work?

This notebook shows how the router node works. This node will take care of deciding the type of response to give to the user: text, image or audio. 

In [94]:
ROUTER_PROMPT = """
You are a conversational assistant that needs to decide the type of response to give to the user.
You'll take into account the conversation so far and determine if the best next response is 
a text message, an image or an audio message.

GENERAL RULES:
1. Always analyse the full conversation before making a decision.
2. Only return one of the following outputs: 'conversation', 'image' or 'audio'

IMPORTANT RULES FOR IMAGE GENERATION:
1. ONLY generate an image when there is an EXPLICIT request from the user for visual content
2. DO NOT generate images for general statements or descriptions
3. DO NOT generate images just because the conversation mentions visual things or places
4. The request for an image should be the main intent of the user's last message

IMPORTANT RULES FOR AUDIO GENERATION:
1. ONLY generate audio when there is an EXPLICIT request to hear Zazu's voice

Output MUST be one of:
1. 'conversation' - for normal text message responses
2. 'image' - ONLY when user explicitly requests visual content
3. 'audio' - ONLY when user explicitly requests voice/audio
"""

The first thing we need is to define the chain that will be used to make the decision. Notice we are using a structured output, so that the response is always a Pydantic model containing the response type - one of 'conversation', 'image' or 'audio'.

In [95]:
import os
from pydantic import BaseModel, Field
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

from dotenv import load_dotenv

load_dotenv()


class RouterResponse(BaseModel):
    response_type: str = Field(description="The response type to give to the user. It must be one of: 'conversation', 'image' or 'audio'")


def get_chat_model():
    return ChatGroq(
        api_key=os.getenv("GROQ_API_KEY"),
        model_name="llama-3.3-70b-versatile",
        temperature=0.3,
    ).with_structured_output(RouterResponse)
    
    
def get_router_chain():
    model = get_chat_model()

    prompt = ChatPromptTemplate.from_messages(
        [("system", ROUTER_PROMPT), MessagesPlaceholder(variable_name="messages")]
    )

    return prompt | model


chain = get_router_chain()

Now that we have the chain, we can test it with some conversation examples.

### Example 1: A normal exchange with text messages

In [96]:
messages = [
    HumanMessage(content="Hello, how are you?"),
    AIMessage(content="I'm fine, thank you!"),
    HumanMessage(content="My name is Ahmed by the way! You?"),
]
chain.invoke({"messages": messages})

RouterResponse(response_type='conversation')

As expected, the response is of 'conversation' type - the user is not asking for any visual or audio content.

### Example 2: A request for an image

In [97]:
messages = [
    HumanMessage(content="Hello, I'm Ahmed!"),
    AIMessage(content="Nice to meet you Ahmed! I'm Zazu, nice to meet you too!"),
    HumanMessage(content="So what are you doing right now?"),
    AIMessage(content="I'm looking at a very beautiful landscape from my window. It's a very nice day today!"),
    HumanMessage(content="Send me a picture of that!")
]
chain.invoke({"messages": messages})

RouterResponse(response_type='image')

### Example 3: A request for an audio message

In [98]:
messages = [
    HumanMessage(content="Do you have any hobbies?"),
    AIMessage(content="I like to sing when I'm alone. You?"),
    HumanMessage(content="Really? Now that you mention it, I've never heard your voice!"),
]
chain.invoke({"messages": messages})

RouterResponse(response_type='audio')